In [ ]:
import time

In [ ]:
import json

In [ ]:
from datetime import datetime

In [ ]:
import nexussdk as nxs

In [ ]:
# Example:
# https://nexus.domain.org/v1
with open("DEPLOYMENT", "r") as f:
    DEPLOYMENT = f.readline().rstrip()
    nxs.config.set_environment(DEPLOYMENT)

In [ ]:
with open("TOKEN", "r") as f:
    TOKEN = f.readline().rstrip()
    nxs.config.set_token(TOKEN)

In [ ]:
pp = nxs.tools.pretty_print

# Examples setup

In [ ]:
def utcnow() -> str:
    return datetime.utcnow().strftime("%Y%m%d%H%M%S%f")

In [ ]:
def generate_compacted_id() -> str:
    return "nxv:storage{}".format(utcnow())

In [ ]:
def generate_expanded_id() -> str:
    return "{}/storage{}".format("https://knowledgegraph.domain.org", utcnow())

In [ ]:
ORGANIZATION = "myorg{}".format(utcnow())

In [ ]:
nxs.organizations.create(ORGANIZATION)["@id"]

In [ ]:
PROJECT = "myproject"

In [ ]:
nxs.projects.create(ORGANIZATION, PROJECT)["@id"]

In [ ]:
with open("S3_ENDPOINT") as f:
    S3_ENDPOINT = f.readline().rstrip()

In [ ]:
with open("S3_ACCESS_KEY") as f:
    S3_ACCESS_KEY = f.readline().rstrip()

In [ ]:
with open("S3_SECRET_KEY") as f:
    S3_SECRET_KEY = f.readline().rstrip()

In [ ]:
with open("EXTERNAL_DISK_ENDPOINT") as f:
    EXTERNAL_DISK_ENDPOINT = f.readline().rstrip()

In [ ]:
with open("EXTERNAL_DISK_CREDENTIALS") as f:
    EXTERNAL_DISK_CREDENTIALS = f.readline().rstrip()

### User API

In [ ]:
s3_storage_id = nxs.storages.create_s3_storage(
    ORGANIZATION,
    PROJECT,
    "sdkbucket",
    endpoint = S3_ENDPOINT,
    access_key = S3_ACCESS_KEY,
    secret_key = S3_SECRET_KEY)["@id"]

In [ ]:
disk_storage_id = nxs.storages.create_disk_storage(ORGANIZATION, PROJECT, "/tmp")["@id"]

In [ ]:
external_disk_storage_id = nxs.storages.create_external_disk_storage(
    ORGANIZATION,
    PROJECT,
    EXTERNAL_DISK_ENDPOINT,
    "testproject",
    credentials=EXTERNAL_DISK_CREDENTIALS)["@id"]

In [ ]:
pp(nxs.storages.fetch(ORGANIZATION, PROJECT, s3_storage_id))

In [ ]:
pp(nxs.storages.fetch(ORGANIZATION, PROJECT, disk_storage_id))

In [ ]:
pp(nxs.storages.fetch(ORGANIZATION, PROJECT, external_disk_storage_id))

In [ ]:
pp(nxs.storages.list(ORGANIZATION, PROJECT))

In [ ]:
pp(nxs.storages.list(ORGANIZATION, PROJECT, type="nxv:S3Storage"))

In [ ]:
pp(nxs.storages.tag(ORGANIZATION, PROJECT, disk_storage_id, tag="v1.1.1", rev_to_tag=1, rev=1))

In [ ]:
pp(nxs.storages.tags(ORGANIZATION, PROJECT, disk_storage_id))

## Files API

In [ ]:
linked_image = nxs.files.create_link(ORGANIZATION, PROJECT, "idontalways.jpg", "6emgw.jpg", "image/jpeg", storage_id=s3_storage_id)

In [ ]:
pp(linked_image)

In [ ]:
pp(nxs.files.fetch(ORGANIZATION, PROJECT, linked_image["@id"], out_filepath = linked_image["_filename"]))

![I don't always](idontalways.jpg)

In [ ]:
pp(nxs.files.create(ORGANIZATION, PROJECT, "idontalways.jpg"))

In [ ]:
pp(nxs.files.update_link(ORGANIZATION, PROJECT, "idontalways.jpg", "6emgw.jpg", "image/jpeg", 1, file_id=linked_image["@id"], storage_id=s3_storage_id))

In [ ]:
pp(nxs.files.list(ORGANIZATION, PROJECT))

In [ ]:
pp(nxs.files.tag(ORGANIZATION, PROJECT, linked_image["@id"], 1, "original", rev=2))

In [ ]:
pp(nxs.files.fetch(ORGANIZATION, PROJECT, linked_image["@id"], tag="original"))

In [ ]:
pp(nxs.files.deprecate(ORGANIZATION, PROJECT, linked_image["@id"], 3))